In [59]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

## Import dependencies

In [61]:
import os
import pandas as pd
from IPython.display import display

from configs import config
from utils.common_helpers import create_tuned_base_model
from utils.data_loader import ACSEmploymentDataset
from utils.generic_pipeline import GenericPipeline
from utils.generic_analyzer import GenericAnalyzer

## Configs

In [62]:
STATE = config.DATASET_CONFIG['state']
YEAR = config.DATASET_CONFIG['year']
DATASET_NAME = f"Folktables_{STATE}_{YEAR}"

SEX_priv = RACE_priv = str(1)
PROTECTED_GROUPS = ['SEX','RAC1P']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('..', '..', 'results', 'models_tuning', 'tuning_results_Folktables_GA_2018_20221215__105658.csv')

## Models tuned hyper-parameters

In [63]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Unnamed: 0,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,0,Folktables_GA_2018,LogisticRegression,0.8117,0.8122,"{'max_iter': 50, 'penalty': 'l2', 'solver': 'l..."
1,1,Folktables_GA_2018,DecisionTreeClassifier,0.8228,0.8230,"{'criterion': 'entropy', 'max_depth': 10, 'max..."
2,2,Folktables_GA_2018,RandomForestClassifier,0.8292,0.8295,"{'max_depth': 10, 'max_features': 0.6, 'min_sa..."
3,3,Folktables_GA_2018,XGBClassifier,0.8313,0.8318,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti..."
4,4,Folktables_GA_2018,KNeighborsClassifier,0.8063,0.8068,"{'metric': 'manhattan', 'n_neighbors': 15, 'we..."


## Preprocess dataset

In [64]:
dataset = ACSEmploymentDataset(state=[STATE], year=YEAR, root_dir=os.path.join('..', '..', 'data'), with_nulls=False)
dataset.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,3,2,1,2,2,2,1,2,16,1,1,13,51
1,3,4,0,1,2,1,1,2,1,2,1,16,1,4,16,56
2,5,4,0,1,1,1,1,2,2,2,2,17,1,4,20,23
3,1,4,0,1,2,1,2,2,2,1,2,16,1,1,17,43
4,5,4,0,1,2,1,2,2,2,2,1,16,1,1,19,20


## Run experiments

In [65]:
def create_base_pipeline(dataset, protected_groups, priv_values, model_seed):
    base_pipeline = GenericPipeline(dataset, protected_groups, priv_values)
    _ = base_pipeline.create_preprocessed_train_test_split(dataset, config.TEST_SET_FRACTION, seed=model_seed)

    print('\nProtected groups splits:')
    for g in base_pipeline.test_groups.keys():
        print(g, base_pipeline.test_groups[g].shape)

    return base_pipeline


def get_model_metrics(base_model, dataset, protected_groups, priv_values, model_seed,
                      dataset_name, base_model_name, exp_num=1):
    base_pipeline = create_base_pipeline(dataset, protected_groups, priv_values, model_seed)

    base_model.fit(base_pipeline.X_train_val, base_pipeline.y_train_val)
    y_preds = base_model.predict(base_pipeline.X_test)

    analyzer = GenericAnalyzer(base_pipeline.X_test, base_pipeline.y_test,
                               ['SEX','RAC1P'], [SEX_priv, RACE_priv],
                               base_pipeline.test_groups)
    dtc_res = analyzer.compute_metrics(y_preds, base_model_name)
    analyzer.save_metrics_to_file(dataset_name, base_model_name, exp_num=exp_num)

    results_df = pd.DataFrame(dtc_res)
    return results_df


In [66]:
def run_experiment(exp_num, model_seed):
    for model_idx in range(len(config.MODELS_CONFIG)):
        print('#' * 30, f' [Experiment {exp_num}] Analyze {config.MODELS_CONFIG[model_idx]["model_name"]} ', '#' * 30)
        model_seed += 1
        base_model = create_tuned_base_model(config.MODELS_CONFIG[model_idx]['model'],
                                             config.MODELS_CONFIG[model_idx]['model_name'],
                                             models_tuned_params_df)
        results_df = get_model_metrics(base_model, dataset, PROTECTED_GROUPS, PRIV_VALUES,
                                       model_seed=model_seed,
                                       dataset_name=DATASET_NAME,
                                       base_model_name=config.MODELS_CONFIG[model_idx]['model_name'],
                                       exp_num=exp_num)
        print(f'\n[Experiment {exp_num}] Metrics confusion matrix:')
        display(results_df)

        print('\n\n\n')


### Experiment 1

In [67]:
run_experiment(exp_num=1, model_seed=100)

##############################  [Experiment 1] Analyze LogisticRegression  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6609, 16)
SEX_RAC1P_dis (3662, 16)
SEX_priv (9901, 16)
SEX_dis (10270, 16)
RAC1P_priv (13217, 16)
RAC1P_dis (6954, 16)

[Experiment 1] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.832649,0.886848,0.780440,0.883288,0.775414,0.836193,0.825365
TNR,0.785842,0.780080,0.787535,0.789258,0.782947,0.780294,0.795755
PPV,0.758415,0.808097,0.728097,0.796492,0.714441,0.763289,0.748462
FNR,0.167351,0.113152,0.219560,0.116712,0.224586,0.163807,0.174635
FPR,0.214158,0.219920,0.212465,0.210742,0.217053,0.219706,0.204245
Accuracy,0.806752,0.834619,0.784544,0.834663,0.779844,0.805932,0.808312
F1,0.793800,0.845643,0.753360,0.837648,0.743680,0.798079,0.785035
Selection-Rate,0.490457,0.560599,0.451939,0.535501,0.447030,0.502459,0.467645
Positive-Rate,1.097880,1.097453,1.071891,1.108973,1.085343,1.095513,1.102747






##############################  [Experiment 1] Analyze DecisionTreeClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6582, 16)
SEX_RAC1P_dis (3543, 16)
SEX_priv (9817, 16)
SEX_dis (10354, 16)
RAC1P_priv (13393, 16)
RAC1P_dis (6778, 16)

[Experiment 1] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.857143,0.854853,0.862941,0.865376,0.848294,0.848241,0.876521
TNR,0.794157,0.862901,0.759314,0.849588,0.746544,0.795103,0.792420
PPV,0.765932,0.862943,0.725391,0.835223,0.702070,0.774974,0.747558
FNR,0.142857,0.145147,0.137059,0.134624,0.151706,0.151759,0.123479
FPR,0.205843,0.137099,0.240686,0.150412,0.253456,0.204897,0.207580
Accuracy,0.821873,0.858857,0.803274,0.856983,0.788584,0.819234,0.827088
F1,0.808974,0.858879,0.788210,0.850032,0.768286,0.809954,0.806919
Selection-Rate,0.492440,0.497721,0.504657,0.485281,0.499227,0.497051,0.483328
Positive-Rate,1.119085,0.990626,1.189621,1.036103,1.208275,1.094541,1.172513






##############################  [Experiment 1] Analyze RandomForestClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6642, 16)
SEX_RAC1P_dis (3614, 16)
SEX_priv (9811, 16)
SEX_dis (10360, 16)
RAC1P_priv (13388, 16)
RAC1P_dis (6783, 16)

[Experiment 1] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.873247,0.875474,0.874349,0.887526,0.857638,0.863344,0.894959
TNR,0.796563,0.848739,0.772377,0.841763,0.758495,0.794922,0.799544
PPV,0.777679,0.860665,0.739537,0.839264,0.718073,0.785088,0.762462
FNR,0.126753,0.124526,0.125651,0.112474,0.142362,0.136656,0.105041
FPR,0.203437,0.151261,0.227623,0.158237,0.241505,0.205078,0.200456
Accuracy,0.830995,0.862541,0.815717,0.863826,0.799903,0.826710,0.839452
F1,0.822697,0.868006,0.801313,0.862721,0.781675,0.822358,0.823415
Selection-Rate,0.504189,0.525143,0.502490,0.509836,0.498842,0.510905,0.490933
Positive-Rate,1.122888,1.017206,1.182292,1.057505,1.194361,1.099678,1.173775






##############################  [Experiment 1] Analyze XGBClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6371, 16)
SEX_RAC1P_dis (3671, 16)
SEX_priv (9718, 16)
SEX_dis (10453, 16)
RAC1P_priv (13153, 16)
RAC1P_dis (7018, 16)

[Experiment 1] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.865737,0.894720,0.848289,0.895902,0.834433,0.862585,0.872160
TNR,0.809205,0.832606,0.790292,0.840349,0.782695,0.802856,0.820349
PPV,0.783852,0.840511,0.747015,0.832417,0.736008,0.786440,0.778686
FNR,0.134263,0.105280,0.151711,0.104098,0.165567,0.137415,0.127840
FPR,0.190795,0.167394,0.209708,0.159651,0.217305,0.197144,0.179651
Accuracy,0.834317,0.863444,0.814764,0.866433,0.804458,0.830153,0.842120
F1,0.822762,0.866769,0.794438,0.862993,0.782136,0.822755,0.822777
Selection-Rate,0.490605,0.528488,0.479161,0.505351,0.476897,0.501254,0.470647
Positive-Rate,1.104464,1.064496,1.135571,1.076266,1.133728,1.096822,1.120041






##############################  [Experiment 1] Analyze KNeighborsClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6476, 16)
SEX_RAC1P_dis (3601, 16)
SEX_priv (9790, 16)
SEX_dis (10381, 16)
RAC1P_priv (13256, 16)
RAC1P_dis (6915, 16)

[Experiment 1] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.841386,0.881587,0.821192,0.880351,0.799353,0.838315,0.847917
TNR,0.775161,0.797101,0.751793,0.800117,0.754050,0.773868,0.777447
PPV,0.751041,0.818847,0.704946,0.800818,0.699394,0.760895,0.731138
FNR,0.158614,0.118413,0.178808,0.119649,0.200647,0.161685,0.152083
FPR,0.224839,0.202899,0.248207,0.199883,0.245950,0.226132,0.222553
Accuracy,0.804720,0.840179,0.780894,0.838407,0.772951,0.803636,0.806797
F1,0.793651,0.849059,0.758642,0.838703,0.746040,0.797731,0.785209
Selection-Rate,0.500025,0.548950,0.488475,0.524617,0.476833,0.508902,0.483008
Positive-Rate,1.120293,1.076620,1.164901,1.099315,1.142923,1.101748,1.159722


### Experiment 2

In [68]:
run_experiment(exp_num=2, model_seed=200)

##############################  [Experiment 2] Analyze LogisticRegression  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6538, 16)
SEX_RAC1P_dis (3565, 16)
SEX_priv (9822, 16)
SEX_dis (10349, 16)
RAC1P_priv (13322, 16)
RAC1P_dis (6849, 16)

[Experiment 2] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.844023,0.898191,0.801061,0.897001,0.787535,0.843632,0.844858
TNR,0.787434,0.794934,0.794847,0.796552,0.779546,0.782159,0.797000
PPV,0.758847,0.813385,0.741104,0.795376,0.718757,0.764179,0.747748
FNR,0.155977,0.101809,0.198939,0.102999,0.212465,0.156368,0.155142
FPR,0.212566,0.205066,0.205153,0.203448,0.220454,0.217841,0.203000
Accuracy,0.812454,0.846436,0.797475,0.843616,0.782878,0.810164,0.816908
F1,0.799172,0.853687,0.769917,0.843137,0.751575,0.801942,0.793342
Selection-Rate,0.491746,0.550780,0.457223,0.528406,0.456952,0.502927,0.469996
Positive-Rate,1.112245,1.104262,1.080902,1.127771,1.095690,1.103971,1.129870






##############################  [Experiment 2] Analyze DecisionTreeClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6584, 16)
SEX_RAC1P_dis (3677, 16)
SEX_priv (9883, 16)
SEX_dis (10288, 16)
RAC1P_priv (13195, 16)
RAC1P_dis (6976, 16)

[Experiment 2] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.850873,0.872593,0.824127,0.879831,0.819068,0.847324,0.858262
TNR,0.802247,0.837021,0.782588,0.827894,0.780157,0.805195,0.797080
PPV,0.777710,0.849193,0.739601,0.824624,0.728793,0.789570,0.754417
FNR,0.149127,0.127407,0.175873,0.120169,0.180932,0.152676,0.141738
FPR,0.197753,0.162979,0.217412,0.172106,0.219843,0.194805,0.202920
Accuracy,0.824054,0.855255,0.800381,0.852777,0.796462,0.824706,0.822821
F1,0.812648,0.860734,0.779580,0.851333,0.771298,0.817428,0.802996
Selection-Rate,0.490655,0.526731,0.477291,0.511181,0.470937,0.497006,0.478641
Positive-Rate,1.094075,1.027556,1.114286,1.066948,1.123869,1.073147,1.137649






##############################  [Experiment 2] Analyze RandomForestClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6494, 16)
SEX_RAC1P_dis (3561, 16)
SEX_priv (9812, 16)
SEX_dis (10359, 16)
RAC1P_priv (13292, 16)
RAC1P_dis (6879, 16)

[Experiment 2] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.877959,0.882605,0.874425,0.889831,0.865411,0.872379,0.889902
TNR,0.795274,0.851691,0.757353,0.847409,0.750042,0.793545,0.798361
PPV,0.773994,0.857949,0.728767,0.837423,0.715126,0.782053,0.757612
FNR,0.122041,0.117395,0.125575,0.110169,0.134589,0.127621,0.110098
FPR,0.204726,0.148309,0.242647,0.152591,0.249958,0.206455,0.201639
Accuracy,0.831987,0.867262,0.807357,0.867305,0.798533,0.829747,0.836313
F1,0.822705,0.870102,0.794979,0.862832,0.783124,0.824750,0.818446
Selection-Rate,0.503644,0.518171,0.512496,0.498369,0.508640,0.512263,0.486989
Positive-Rate,1.134323,1.028737,1.199869,1.062581,1.210152,1.115498,1.174614






##############################  [Experiment 2] Analyze XGBClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6587, 16)
SEX_RAC1P_dis (3569, 16)
SEX_priv (9879, 16)
SEX_dis (10292, 16)
RAC1P_priv (13310, 16)
RAC1P_dis (6861, 16)

[Experiment 2] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.869789,0.900517,0.852362,0.900564,0.836962,0.867456,0.874780
TNR,0.801280,0.824848,0.787286,0.828371,0.777369,0.796237,0.810353
PPV,0.776433,0.836631,0.749135,0.820898,0.730987,0.781713,0.765466
FNR,0.130211,0.099483,0.147638,0.099436,0.163038,0.132544,0.125220
FPR,0.198720,0.175152,0.212714,0.171629,0.222631,0.203763,0.189647
Accuracy,0.831590,0.862608,0.815074,0.862031,0.802371,0.828775,0.837050
F1,0.820464,0.867399,0.797422,0.858888,0.780393,0.822356,0.816481
Selection-Rate,0.495613,0.537119,0.485850,0.511489,0.480373,0.506987,0.473546
Positive-Rate,1.120238,1.076361,1.137795,1.097047,1.144975,1.109686,1.142807






##############################  [Experiment 2] Analyze KNeighborsClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)

Protected groups splits:
SEX_RAC1P_priv (6625, 16)
SEX_RAC1P_dis (3633, 16)
SEX_priv (9838, 16)
SEX_dis (10333, 16)
RAC1P_priv (13325, 16)
RAC1P_dis (6846, 16)

[Experiment 2] Metrics confusion matrix:


,overall,SEX_RAC1P_priv,SEX_RAC1P_dis,SEX_priv,SEX_dis,RAC1P_priv,RAC1P_dis
TPR,0.838437,0.879134,0.816901,0.874396,0.799080,0.838678,0.837922
TNR,0.771881,0.807357,0.728634,0.800473,0.747618,0.780056,0.757107
PPV,0.751722,0.829376,0.694233,0.804289,0.697152,0.768685,0.717866
FNR,0.161563,0.120866,0.183099,0.125604,0.200920,0.161322,0.162078
FPR,0.228119,0.192643,0.271366,0.199527,0.252382,0.219944,0.242893
Accuracy,0.801943,0.844377,0.766584,0.836247,0.769283,0.807355,0.791411
F1,0.792715,0.853530,0.750588,0.837879,0.744644,0.802158,0.773261
Selection-Rate,0.503793,0.546717,0.505918,0.526123,0.482532,0.508068,0.495472
Positive-Rate,1.115355,1.059994,1.176697,1.087167,1.146207,1.091056,1.167240
